***WMC via Knowledge compilation***


---


1. Write a method that transforms a formula in sd-DNNF form, and use
this method for computing the weighted model count of propositional
formulas.

2. Check the correctness of your algorithm by comparing the results of
your method with the explicit computation of weighted model
counting via truth table.

3. Write a method that estimates #SAT using the sampleSat algorithm,
and compare the result of the approximated counting with the result
obtained by the exact counting.






In [533]:
#import libraries
from sympy import symbols, simplify, Not, And, Or, Implies, satisfiable, S, simplify, sympify
from sympy import symbols, And, Or, Not, Implies
from sympy.logic.boolalg import to_nnf, Boolean
from sympy.logic.inference import satisfiable
from sympy.logic.boolalg import to_cnf
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from itertools import combinations
from functools import reduce
import random
import time

In [534]:
from IPython.display import display, HTML

def render_pipeline(stages):
    # stages: list[(name, done)]
    badges = []
    for name, done in stages:
        icon = "✅" if done else "⏳"
        opacity = "1" if done else ".6"
        badges.append(f"""
        <div style="display:flex; align-items:center; gap:8px; padding:8px 12px;
                    border:1px solid #e5e7eb; border-radius:999px; background:#ffffff; margin:6px;
                    font-family:Inter, Arial; opacity:{opacity};">
            <span style="width:10px; height:10px; border-radius:999px; background:{'#10b981' if done else '#fbbf24'}; display:inline-block;"></span>
            <span style="font-size:14px;">{icon}</span>
            <span style="font-weight:600; font-size:14px;">{name}</span>
        </div>""")
    n_done = sum(1 for _, d in stages if d)
    total = len(stages) or 1
    pct = int(100 * n_done / total)
    bar = f"""
    <div style="height:10px; background:#f0f0f0; border-radius:6px; overflow:hidden; margin-top:6px;">
        <div style="height:10px; width:{pct}%; background:#10b981;"></div>
    </div>"""
    return f"""
    <div>
        <div style="display:flex; flex-wrap:wrap;">{''.join(badges)}</div>
        <div style="font-family:Inter, Arial; margin:6px 2px; font-size:13px;">Progress: {pct}%</div>
        {bar}
    </div>"""

In [535]:
from IPython.display import display, HTML

def render_kpi_cards(results):
    # results: list of dicts with keys: name, exact, approx
    cards = []
    for r in results:
        exact = float(r.get("exact", 0))
        calculated = float(r.get("calculated", 0))
        delta = calculated - exact
        rel = (abs(delta) / exact) if exact != 0 else float("inf")

        def fmt_int(x):
            try:
                x = int(x)
                return f"{x:,}".replace(",", ".")
            except Exception:
                return str(x)

        def fmt_float(x):
            try:
                return f"{x:.4f}"
            except Exception:
                return str(x)

        cards.append(f"""
        <div style="flex:1; min-width:260px; max-width:360px;
                    border:1px solid #e0dede; border-radius:18px; padding:18px; margin:12px;
                    background:linear-gradient(135deg, #fef6f9, #f3f8ff);
                    font-family:Inter, Arial, sans-serif; color:#333; 
                    box-shadow:0 4px 10px rgba(0,0,0,0.05);">
            <div style="font-size:14px; color:#666; font-weight:500;">Test</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('name','N/A')}
            </div>
            <div style="font-size:14px; color:#666; font-weight:500;">Formula</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('formula','N/A')}
            </div>
            <div style="font-size:14px; color:#666; font-weight:500;">sd-DNNF</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('sd-DNNF','N/A')}
            </div>
            <div style="display:flex; gap:14px; align-items:baseline; margin:6px 0;">
                <div style="flex:1; background:#d9f2e6; color:#224; border-radius:12px; padding:10px;">
                    <div style="font-size:12px; opacity:.8;">Exact</div>
                    <div style="font-size:20px; font-weight:700;">{fmt_int(exact)}</div>
                </div>
                <div style="flex:1; background:#ffe6ea; color:#224; border-radius:12px; padding:10px;">
                    <div style="font-size:12px; opacity:.8;">Calculated</div>
                    <div style="font-size:20px; font-weight:700;">{fmt_int(calculated)}</div>
                </div>
            </div>
            <div style="display:flex; gap:14px; margin-top:12px;">
                <div style="flex:1; border:1px dashed #ccc; border-radius:12px; padding:10px; background:#f7fdfb;">
                    <div style="font-size:12px; color:#666;">Δ (calculated - exact)</div>
                    <div style="font-weight:700; color:#333;">{fmt_int(delta)}</div>
                </div>
            </div>
        </div>
        """)

    html = f"""
    <div style="display:flex; flex-wrap:wrap; gap:10px; align-items:stretch;">
        {''.join(cards)}
    </div>
    """
    display(HTML(html))

In [536]:
# ===========================
# Styling pastel
# ===========================
def style_pastel(df: pd.DataFrame):
    df = df.drop(columns=['relative_error','p_hat','n'], errors='ignore')
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    sty = (df.style
           .hide(axis="index")
           .set_table_styles([
               {"selector": "th.col_heading",
                "props": [("background", "#f0f0f0"), ("color", "#374151"),
                          ("font-size", "13px"), ("font-weight", "600"),
                          ("padding", "10px 8px"), ("border", "none")]},
               {"selector": "tbody td",
                "props": [("padding", "10px 12px"), ("border-bottom", "1px solid #e5e7eb"),
                          ("font-size", "13px"), ("color", "#f0f0f0")]},
               {"selector": "table",
                "props": [("border-collapse", "separate"), ("border-spacing", "0"),
                          ("border", "1px solid #e5e7eb"), ("border-radius", "12px"),
                          ("overflow", "hidden"), ("box-shadow", "0 4px 12px rgba(0,0,0,.05)")]},
           ])
           .format(precision=4)
          )

    if num_cols:
        # Gradiente pastello (viola→rosa→azzurro chiaro)
        sty = sty.background_gradient(
            cmap="coolwarm", subset=num_cols,
            vmin=df[num_cols].min().min(),
            vmax=df[num_cols].max().max()
        )

    return sty

Pipeline to convert a formula F into sd-DNNF:
1.	Start with a formula **F**.
2.	Transform it into ***NNF*** (Negation Normal Form): a formula where negations appear only in front of atomic propositions.
3.	Transform it into ***DNNF*** (Decomposable Negation Normal Form): an NNF where each conjunction shares no atomic propositions, i.e., Φ₁ ∧ Φ₂ such that props(Φ₁) ∩ props(Φ₂) = ∅. To achieve this, Shannon’s Expansion is applied.
4.	Transform it into ***d-DNNF*** (Deterministic Decomposable Negation Normal Form): a DNNF where, for every disjunction Φ₁ ∨ Φ₂ ∨ … ∨ Φₙ in the formula, there is at most one i such that I |= Φᵢ.
This ensures that the branches of an OR are mutually exclusive; otherwise, in Weighted Model Counting (WMC), we risk double-counting the same truth assignment.
5.	Transform it into ***sd-DNNF*** (Smooth Deterministic Decomposable Negation Normal Form): apply smoothing (left and right) to guarantee that all disjuncts contain all propositional variables — see the transformation rules (in practice, a tautology is introduced for missing atomic propositions).

Note: Strictly speaking, the very first step would be the transformation of F into CNF (Conjunctive Normal Form): a formula in which implication (→) and equivalence (≡) symbols do not appear. However, in our code this step is handled by the to_nnf function (imported).

In [537]:
# ============================================================
#  F -> NNF -> DNNF -> d-DNNF -> sd-DNNF (con Tau) -> WMC
#  (no normalize_formula: use directly to_nnf)
# ============================================================
# ---------------------------
# Boolean node Tau(v) for visible structural smoothing
# ---------------------------
class Tau(Boolean):
    """
    Represents the tautology (v ∨ ¬v) as an explicit node.
    In WMC, it contributes [w(v) + w(¬v)].
    """
    is_Tau = True

    def __new__(cls, v):
        v = sympify(v)
        obj = Boolean.__new__(cls)
        obj._v = v
        return obj

    @property
    def v(self):
        return self._v

    @property
    def args(self):
        return (self._v,)

    def _hashable_content(self):
        return (self._v,)

    @property
    def free_symbols(self):
        return {self._v}

def _tau(v):
    return Tau(v)

def _vars(e):
    return set(e.free_symbols)

# ---------------------------
# Shannon expansion (NO simplify)
# ---------------------------
def shannon_expansion(f, s):
    """
    f == (s ∧ f|s=True) ∨ (¬s ∧ f|s=False)
    """
    if s not in f.free_symbols:
        return f
    f1 = f.subs({s: S.true})
    f0 = f.subs({s: S.false})
    return Or(And(s, f1), And(Not(s), f0))

# ---------------------------
# Utilities: variable-connected components
# ---------------------------
def _var_components(conj):
    """
    Group the factors of an AND into variable-connected blocks.
    Different blocks do not share variables -> decomposable AND.
    """
    if isinstance(conj, And):
        factors = list(conj.args)
    else:
        factors = [conj]
    comps = []
    while factors:
        seed = factors.pop()
        group = [seed]
        vars_grp = set(seed.free_symbols)
        changed = True
        while changed:
            changed = False
            rest = []
            for f in factors:
                if vars_grp & set(f.free_symbols):
                    group.append(f)
                    vars_grp |= set(f.free_symbols)
                    changed = True
                else:
                    rest.append(f)
            factors = rest
        comps.append(group)
    return comps

# ---------------------------
# 1) NNF -> DNNF
# Function that takes a NNF and returns a DNNF.
# It applies Shannon's Expansion in order to ensure decomposability.
# Every conjunction does not have common atomic propositions, i.e. Φ₁ ∧ Φ₂ such that props(Φ₁) ∩ (Φ₂) = ∅.
# ---------------------------
def NNF2DNNF(nnf):

    #Take Boolean variables from the formula and put them in a list
    #nnf.free_symbols is a set of SymPy symbols that appear in the expression
    atoms = list(nnf.free_symbols)

    #base case: the formula has 0 or 1 variable
    if len(atoms) <= 1:
        return simplify(nnf)

    #I build groups that are “variable-connected”: if I have only one group → not decomposable ⇒ I apply Shannon
    if isinstance(nnf, And): #check if current node is conjunction
        # _var_components takes the factors of the AND (e.g., F1 ∧ F2 ∧ F3) and groups 
        # Them into blocks such that within each block the factors share variables. 
        # If different blocks do not share variables → they are candidates for a decomposable AND.
        # Example: (A∨B)∧(¬A∨C)∧(D∨E) produces groups = [[A∨B, ¬A∨C], [D∨E]].
        groups = _var_components(nnf)
        if len(groups) > 1:
            compiled = [
                NNF2DNNF(And(*g)) if len(g) > 1 else NNF2DNNF(g[0])
                for g in groups ]
            return simplify(And(*compiled))  # decomposable AND 
        else: # No possible decomposition -> force split with Shannon
            pivot = atoms[0]
            return NNF2DNNF(shannon_expansion(nnf, pivot))

    if isinstance(nnf, Or):
        kids = [NNF2DNNF(k) for k in nnf.args]
        return simplify(Or(*kids))          # determinism after
    
    # fallback
    pivot = atoms[0]
    return NNF2DNNF(shannon_expansion(nnf, pivot))

# ---------------------------
# 2) DNNF -> d-DNNF (determinism on OR)
# ---------------------------
def _make_or_deterministic(expr):
    """
    Makes Or(...) deterministic without infinite recursions.
    Strategy:
    - as long as there is a joinable pair of children,
    choose a pivot in the INTERSECTION of their variables (if empty, in the UNION),
    apply Shannon to the entire OR and restart. Iterative.
    """
    assert isinstance(expr, Or)
    # flatten + dedup
    args = list(Or(*expr.args).args)
    uniq = []
    for a in args:
        if a not in uniq:
            uniq.append(a)
    args = uniq
    cur = Or(*args)

    while True:
        n = len(args)
        changed = False

        for i in range(n):
            for j in range(i+1, n):
                if satisfiable(And(args[i], args[j])):  # overlap -> not deterministic
                    Vi, Vj = _vars(args[i]), _vars(args[j])
                    cand = list(Vi & Vj) or list(Vi | Vj)  # first intersection, then union
                    for p in cand:
                        new_or = shannon_expansion(Or(*args), p)
                        if new_or != cur:       # check structural progress
                            cur = new_or
                            args = list(cur.args) if isinstance(cur, Or) else [cur]
                            changed = True
                            break
                    break
            if changed:
                break
        if not changed:
            return cur

# ---------------------------
# 2) DNNF -> d-DNNF (determinism on OR)
# ---------------------------
def DNNF2dDNNF(dnnf):
    if isinstance(dnnf, And):
        return And(*(DNNF2dDNNF(a) for a in dnnf.args))
    if isinstance(dnnf, Or):
        kids = [DNNF2dDNNF(a) for a in dnnf.args]
        det = _make_or_deterministic(Or(*kids))
        if isinstance(det, Or):
            return Or(*(DNNF2dDNNF(a) for a in det.args))
        return det
    #print("dnnf: ", dnnf)
    return dnnf  # literal / True / False

# ---------------------------
# 3) d-DNNF -> sd-DNNF (smoothing with Tau)
# ---------------------------
def ddnnf2sdNNF(ddnnf):
    if isinstance(ddnnf, And):
        return And(*(ddnnf2sdNNF(a) for a in ddnnf.args))
    if isinstance(ddnnf, Or):
        kids = [ddnnf2sdNNF(a) for a in ddnnf.args]
        allv = set().union(*(_vars(k) for k in kids))
        smoothed = []
        for k in kids:
            missing = allv - _vars(k)
            kk = k
            for v in missing:
                kk = And(kk, _tau(v))  # keep Tau(v) visible
            smoothed.append(kk)
        return Or(*smoothed)
    #print("ddnnf: ", ddnnf)
    return ddnnf  # literal / True / False

# ---------------------------
# 4) WMC on sd-DNNF (weights to literals)
# ---------------------------
def model_counting_sdnnf(sdNNF_formula, weights):
    """
    Calculate WMC on sd-DNNF:
      - literal: weights[literal]
      - Tau(v):  weights[v] + weights[Not(v)]
      - AND:     products of children
      - OR:      sum of children
    """
    if sdNNF_formula is S.true:
        return 1.0
    if sdNNF_formula is S.false:
        return 0.0

    # literals (a, ~a, ...)
    if sdNNF_formula in weights:
        #print("sdNNF_formula, ", float(weights[sdNNF_formula]))
        return float(weights[sdNNF_formula])

    # Tau(v) inserted from smoothing
    if isinstance(sdNNF_formula, Tau):
        v = sdNNF_formula.v
        return float(weights[v] + weights[Not(v)])

    # AND: product
    if isinstance(sdNNF_formula, And):
        val = 1.0
        for ch in sdNNF_formula.args:
            val *= model_counting_sdnnf(ch, weights)
        return val

    # OR: sum
    if isinstance(sdNNF_formula, Or):
        return sum(model_counting_sdnnf(ch, weights) for ch in sdNNF_formula.args)

    raise ValueError(f"Unmanaged node: {sdNNF_formula}")

# ---------------------------
# 5) Orchestrator End-to-end: F -> sd-DNNF
# ---------------------------
def compile_to_sdDNNF(F):
    """
    F -> NNF -> DNNF -> d-DNNF -> sd-DNNF
    (no final simplify to avoid losing the Tau nodes)
    """
    # removes ⇒, ↔ e force NOT to literals
    
    F_nnf  = to_nnf(F, simplify=True)   
    print("to_nnf: ", F_nnf)
    time.sleep(2.5)
    stages[0] = ("to NNF", True); handle.update(HTML(render_pipeline(stages)))

    F_dnnf = NNF2DNNF(F_nnf)
    print("F_dnnf: ", F_dnnf)
    time.sleep(2)
    stages[1] = ("to DNNF", True); handle.update(HTML(render_pipeline(stages)))
    
    F_dd   = DNNF2dDNNF(F_dnnf)
    print("F_dd: ", F_dd)
    time.sleep(2)
    stages[2] = ("to dDNNF", True); handle.update(HTML(render_pipeline(stages)))
    
    F_sdd  = ddnnf2sdNNF(F_dd)
    print("F_sdd: ", F_sdd)
    time.sleep(2)
    stages[3] = ("to sdNNF", True); handle.update(HTML(render_pipeline(stages)))
    return F_sdd

For other test cases -> modify the formula and weights section

In [555]:
# ==============================================================
# SampleSAT and comparison with estimate #SAT vs #SAT via sd-DNNF
# ==============================================================

# --- utilities ---
def _vars_set(F):
    if "_vars" in globals() and callable(_vars):
        return set(_vars(F))
    return set(F.free_symbols)

def _eval_under(F, assign):
    return bool(F.subs(assign))

def _random_assignment(V, rng=None):
    rng = rng or random
    return {v: bool(rng.getrandbits(1)) for v in V}

def _cnf_clauses(F):
    cnf = to_cnf(F, simplify=True)
    if cnf == True:
        return []
    if cnf == False:
        return [[]]
    if isinstance(cnf, Or):
        return [[arg for arg in cnf.args]]
    if isinstance(cnf, And):
        clauses = []
        for c in cnf.args:
            if isinstance(c, Or):
                clauses.append(list(c.args))
            else:
                clauses.append([c])
        return clauses
    return [[cnf]]

def _lit_is_true(lit, assign):
    if isinstance(lit, Not):
        v = lit.args[0]
        return assign.get(v, False) is False
    else:
        v = lit
        return assign.get(v, False) is True

def _unsat_clauses(clauses, assign):
    return [cl for cl in clauses if not any(_lit_is_true(l, assign) for l in cl)]

def _flip(var, assign):
    assign[var] = not assign[var]

def _num_unsat(clauses, assign):
    return sum(1 for cl in clauses if not any(_lit_is_true(l, assign) for l in cl))

# --- SampleSAT ---
def sample_sat(F, max_tries=50, max_flips=1000, noise=0.5, rng=None):
    rng = rng or random
    V = _vars_set(F)
    clauses = _cnf_clauses(F)

    if clauses == []:  # tautology
        return True, _random_assignment(V, rng)
    if clauses == [[]]:  # contradiction
        return False, {}

    for _ in range(max_tries):
        A = _random_assignment(V, rng)
        for _ in range(max_flips):
            if _eval_under(F, A):
                return True, dict(A)

            unsat = _unsat_clauses(clauses, A)
            if not unsat:
                return True, dict(A)
            cl = rng.choice(unsat)

            candidate_vars = []
            for lit in cl:
                v = lit.args[0] if isinstance(lit, Not) else lit
                candidate_vars.append(v)

            if rng.random() < noise:
                v_star = rng.choice(candidate_vars)
            else:
                best_v, best_score = None, None
                for v in set(candidate_vars):
                    _flip(v, A)
                    score = _num_unsat(clauses, A)
                    _flip(v, A)
                    if best_score is None or score < best_score:
                        best_score, best_v = score, v
                v_star = best_v if best_v is not None else rng.choice(candidate_vars)

            _flip(v_star, A)

    return False, {}

# --- Estimate #SAT ---
def estimate_sat_count_samplesat(F, runs=200, **samplesat_kwargs):
    V = _vars_set(F)
    n = len(V)
    success = 0
    for _ in range(runs):
        ok, _A = sample_sat(F, **samplesat_kwargs)
        if ok:
            success += 1
    p_hat = success / runs if runs else 0.0
    return p_hat * (2 ** n), {"p_hat": p_hat, "n": n, "success": success, "runs": runs}

def estimate_sat_count_mc(F, runs=1000, rng=None):
    rng = rng or random
    V = _vars_set(F)
    n = len(V)
    success = 0
    for _ in range(runs):
        A = _random_assignment(V, rng)
        if _eval_under(F, A):
            success += 1
    p_hat = success / runs
    return p_hat * (2 ** n), {"p_hat": p_hat, "n": n, "success": success, "runs": runs}

# --- Exact counting via sd-DNNF ---
def exact_count_sdnnf(F):
    sd = compile_to_sdDNNF(F)
    V = _vars_set(F)
    weights = {}
    for v in V:
        weights[v] = 1.0
        weights[Not(v)] = 1.0
    return model_counting_sdnnf(sd, weights)

# --- Comparison ---
def compare_exact_vs_samplesat(F, runs=500, max_tries=50, max_flips=1000, noise=0.5):
    
    est, stats = estimate_sat_count_samplesat(
        F, runs=runs, max_tries=max_tries, max_flips=max_flips, noise=noise
    )

    exact = exact_count_sdnnf(F)
    rel_err = abs(est - exact) / exact if exact != 0 else (0.0 if est == 0 else float("inf"))
    return {
        "exact_count": int(exact),
        "estimated_count": int(est),
        "relative_error": float(rel_err),
        **stats
    }

# --- Comparison ---
def compare_exact_vs_samplesat_mc(F, runs=500, max_tries=50, max_flips=1000, noise=0.5):
    #est, stats = estimate_sat_count_samplesat(
    #    F, runs=runs, max_tries=max_tries, max_flips=max_flips, noise=noise
    #)

    est, stats = estimate_sat_count_mc(
        F, runs=runs, rng=None
    )

    exact = exact_count_sdnnf(F)
    rel_err = abs(est - exact) / exact if exact != 0 else (0.0 if est == 0 else float("inf"))
    return {
        "exact_count": int(exact),
        "estimated_count": int(est),
        "relative_error": float(rel_err),
        **stats
    }


# DEMO

In [539]:
# ===========================
# TEST CASE 1 (Example 7.5 dispensa - result = 136)
# ===========================

# Formula: (a & b) | (c & ¬a)
a, b, c = symbols('a b c')
F1 = Or(And(a, b), And(c, Not(a)))
print("Formula:", F1)

# Weights
weights = {
    a: 2,   Not(a): 1,
    b: 5,   Not(b): 3,
    c: 7,   Not(c): 1
}
stages = [("to NNF", False), ("to DNNF", False), ("to dDNNF", False), ("to sdNNF", False), ("Counting", False)]
handle = display(HTML(render_pipeline(stages)), display_id=True)
# Orchestration till sd-DNNF
F1_sdd = compile_to_sdDNNF(F1)
print("\nsd-DNNF:", F1_sdd)

# WMC calculation with actual weights
wmc_val = model_counting_sdnnf(F1_sdd, weights)
time.sleep(2)
stages[4] = ("Counting", True); handle.update(HTML(render_pipeline(stages)))
print("\nWMC:", wmc_val)

# Results
results = [
    {"name": "F1", "formula":F1, "sd-DNNF": F1_sdd, "exact": 136, "calculated": wmc_val}
]
render_kpi_cards(results)

Formula: (a & b) | (c & ~a)


to_nnf:  (a & b) | (c & ~a)
F_dnnf:  (a & b) | (c & ~a)
F_dd:  (a & b) | (c & ~a)
F_sdd:  (a & b & Tau(c)) | (c & ~a & Tau(b))

sd-DNNF: (a & b & Tau(c)) | (c & ~a & Tau(b))

WMC: 136.0


In [540]:
# ===========================
# TEST CASE 2 — (Exercise 119 dispensa - result = 23)
# ===========================

# Formula: (A ∨ B) → (B ∨ C)
A, B, C = symbols('A B C')
F2 = Implies(Or(A, B), Or(B, C))
print("Formula F2:", F2)

# Literals Weights (from screenshot)
weights = {
    A: 1,  Not(A): 2,
    B: 1,  Not(B): 2,
    C: 1,  Not(C): 2,
}

del stages
stages = [("to NNF", False), ("to DNNF", False), ("to dDNNF", False), ("to sdNNF", False), ("Counting", False)]
handle = display(HTML(render_pipeline(stages)), display_id=True)
# Compilazione e WMC
F2_sdd = compile_to_sdDNNF(F2)
print("\nsd-DNNF:", F2_sdd)

wmc_val = model_counting_sdnnf(F2_sdd, weights)
time.sleep(2)
stages[4] = ("Counting", True); handle.update(HTML(render_pipeline(stages)))
print("\nWMC:", wmc_val)

# Results
results = [
    {"name": "F2", "formula":F2, "sd-DNNF": F2_sdd, "exact": 23, "calculated": wmc_val}
]
render_kpi_cards(results)

Formula F2: Implies(A | B, B | C)


to_nnf:  B | C | (~A & ~B)
F_dnnf:  B | C | ~A
F_dd:  B | (~B & (~A | (A & C)))
F_sdd:  (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))

sd-DNNF: (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))

WMC: 23.0


In [556]:
# ===========================
# DEMO: comparison exact counting vs SampleSAT
# ===========================
formule = [
    ("F1: (A and B) or (C and not a)", Or(And(a, b), And(c, Not(a)))),
    ("F2: (A or B) implies (B or C)", Implies(Or(A, B), Or(B, C))),
    ("A or B or C", Or(A, B, C)),
    ("(A or B) and ( not A or C))", And(Or(A, B), Or(Not(A), C))),
]

import pandas as pd
risultati = []
for nome, F in formule:
    res = compare_exact_vs_samplesat(F, runs=40000, max_tries=600, max_flips=5000, noise=0.5)
    #res_mc = compare_exact_vs_samplesat_mc(F, runs=500)
    #risultati.append({"formula": nome, **res, **res_mc})
    risultati.append({"formula": nome, **res})
    #risultati.append({"formula": nome, **res_mc})

df = pd.DataFrame(risultati)
display(style_pastel(df))

to_nnf:  (a & b) | (c & ~a)
F_dnnf:  (a & b) | (c & ~a)
F_dd:  (a & b) | (c & ~a)
F_sdd:  (a & b & Tau(c)) | (c & ~a & Tau(b))
to_nnf:  B | C | (~A & ~B)
F_dnnf:  B | C | ~A
F_dd:  B | (~B & (~A | (A & C)))
F_sdd:  (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))
to_nnf:  A | B | C
F_dnnf:  A | B | C
F_dd:  B | (~B & (A | (C & ~A)))
F_sdd:  (B & Tau(A) & Tau(C)) | (~B & ((A & Tau(C)) | (C & ~A)))
to_nnf:  (A | B) & (C | ~A)
F_dnnf:  (A & C) | (B & ~A)
F_dd:  (A & C) | (B & ~A)
F_sdd:  (A & C & Tau(B)) | (B & ~A & Tau(C))


formula,exact_count,estimated_count,success,runs
F1: (A and B) or (C and not a),4,8,40000,40000
F2: (A or B) implies (B or C),7,8,40000,40000
A or B or C,7,8,40000,40000
(A or B) and ( not A or C)),4,8,40000,40000


In [554]:

import random, math
from dataclasses import dataclass
from typing import Dict, List, Tuple, Set, Optional
from sympy import And, Or, Not
from sympy.logic.boolalg import to_cnf as _sympy_to_cnf

Literal = Tuple[bool, object]  # (is_positive, sympy.Symbol)
Clause = List[Literal]

def _vars_set(F) -> Set[object]:
    return set(F.free_symbols)

def _random_assignment(V: Set[object], rng: Optional[random.Random]=None) -> Dict[object, bool]:
    rng = rng or random
    return {v: bool(rng.getrandbits(1)) for v in V}

def _eval_under(F, A: Dict[object, bool]) -> bool:
    return bool(F.subs(A))

def _cnf_clauses_force(F) -> List[Clause]:
    cnf = _sympy_to_cnf(F, simplify=True, force=True)
    if cnf == True:
        return []
    if cnf == False:
        return [[]]
    def lit(sym):
        if sym.func is Not:
            return (False, sym.args[0])
        return (True, sym)
    if cnf.func is Or:
        return [[lit(arg) for arg in cnf.args]]
    if cnf.func is And:
        return [[lit(l) for l in c.args] if c.func is Or else [lit(c)] for c in cnf.args]
    return [[lit(cnf)]]

def _cnf_clauses(F) -> List[Clause]:
    # Fast path when F already is CNF-like
    def _lit_from_expr(e):
        if e.func is Not and len(e.args) == 1 and e.args[0].is_Symbol:
            return (False, e.args[0])
        if e.is_Symbol:
            return (True, e)
        return None
    def clause_from(e):
        if e.func is Or:
            lits = []
            for a in e.args:
                lit = _lit_from_expr(a)
                if lit is None:
                    return None
                lits.append(lit)
            return lits
        else:
            lit = _lit_from_expr(e)
            if lit is not None:
                return [lit]
            return None
    if F == True:  return []
    if F == False: return [[]]
    if F.func is And:
        parsed = []
        for c in F.args:
            cl = clause_from(c)
            if cl is None:
                break
            parsed.append(cl)
        else:
            return parsed
    else:
        cl = clause_from(F)
        if cl is not None:
            return [cl]
    return _cnf_clauses_force(F)

def _clause_satisfied(clause: Clause, A: Dict[object, bool]) -> bool:
    for is_pos, var in clause:
        val = A[var]
        if (is_pos and val) or ((not is_pos) and (not val)):
            return True
    return False

def _unsat_clauses_idx(clauses: List[Clause], A: Dict[object, bool]) -> List[int]:
    return [i for i, c in enumerate(clauses) if not _clause_satisfied(c, A)]

def _breakcount(var, A: Dict[object, bool], clauses: List[Clause]) -> int:
    b = 0
    Av = A[var]
    for c in clauses:
        sat = False
        satisfied_by_other = False
        for is_pos, v in c:
            val = A[v]
            if (is_pos and val) or ((not is_pos) and (not val)):
                sat = True
                if v != var:
                    satisfied_by_other = True
        if sat and not satisfied_by_other:
            for is_pos, v in c:
                if v == var:
                    lit_is_true = (is_pos and Av) or ((not is_pos) and (not Av))
                    if lit_is_true:
                        b += 1
                    break
    return b

from dataclasses import dataclass
@dataclass
class SampleSATResult:
    sat: bool
    assignment: Dict[object, bool]
    tries_used: int
    flips_used: int
    last_unsat_count: int

def sample_sat(F,
               max_tries: int = 50,
               max_flips: int = 10000,
               noise: float = 0.5,
               rng: Optional[random.Random] = None) -> SampleSATResult:
    rng = rng or random.Random()
    V = _vars_set(F)
    clauses = _cnf_clauses(F)

    if clauses == []:
        A = _random_assignment(V, rng)
        return SampleSATResult(True, A, 0, 0, 0)
    if clauses == [[]]:
        return SampleSATResult(False, {}, 0, 0, len(clauses))

    for t in range(1, max_tries + 1):
        A = _random_assignment(V, rng)
        for f in range(1, max_flips + 1):
            if _eval_under(F, A):
                return SampleSATResult(True, dict(A), t, f, 0)
            unsat_idx = _unsat_clauses_idx(clauses, A)
            c = clauses[rng.choice(unsat_idx)]
            if rng.random() < noise:
                _, v = rng.choice(c)
                A[v] = not A[v]
            else:
                candidates = [v for _, v in c]
                bc = {v: _breakcount(v, A, clauses) for v in candidates}
                zeros = [v for v in candidates if bc[v] == 0]
                if zeros:
                    v_to_flip = rng.choice(zeros)
                else:
                    min_bc = min(bc.values())
                    best = [v for v in candidates if bc[v] == min_bc]
                    v_to_flip = rng.choice(best)
                A[v_to_flip] = not A[v_to_flip]
    last_unsat = len(_unsat_clauses_idx(clauses, A))
    return SampleSATResult(False, dict(A), max_tries, max_flips, last_unsat)

# --------- #SAT Estimation ---------
def _wilson_interval(k, n, z=1.96):
    if n == 0:
        return (0.0, 1.0)
    p = k/n
    denom = 1 + z**2/n
    center = (p + z*z/(2*n)) / denom
    half = (z*((p*(1-p) + z*z/(4*n))/n)**0.5) / denom
    return (max(0.0, center - half), min(1.0, center + half))

def estimate_count_random(F, trials=5000, rng=None):
    rng = rng or random.Random()
    V = list(_vars_set(F))
    if not V:
        sat = _eval_under(F, {})
        return {
            "n_vars": 0, "trials": trials, "k_sat": trials if sat else 0,
            "p_hat": 1.0 if sat else 0.0, "p_ci": (1.0,1.0) if sat else (0.0,0.0),
            "est_count": 1.0 if sat else 0.0, "est_ci": (1.0,1.0) if sat else (0.0,0.0),
            "method": "random_sampling"
        }
    k = 0
    for _ in range(trials):
        A = {v: bool(rng.getrandbits(1)) for v in V}
        if _eval_under(F, A):
            k += 1
    p_hat = k / trials
    lo, hi = _wilson_interval(k, trials)
    two_n = 2 ** len(V)
    return {
        "n_vars": len(V), "trials": trials, "k_sat": k,
        "p_hat": p_hat, "p_ci": (lo, hi),
        "est_count": p_hat * two_n, "est_ci": (lo * two_n, hi * two_n),
        "method": "random_sampling"
    }

def estimate_count_samplesat(F, runs=200, max_tries=30, max_flips=20000, noise=0.5, rng=None):
    rng = rng or random.Random()
    V = list(_vars_set(F))
    if not V:
        sat = _eval_under(F, {})
        return {
            "n_vars": 0, "runs": runs, "hits": runs if sat else 0,
            "success_rate": 1.0 if sat else 0.0, "est_count_naive": 1.0 if sat else 0.0,
            "method": "samplesat_runs"
        }
    hits = 0
    flips = []
    for _ in range(runs):
        res = sample_sat(F, max_tries=max_tries, max_flips=max_flips, noise=noise, rng=rng)
        if res.sat:
            hits += 1
            flips.append(res.flips_used)
    sr = hits / runs
    two_n = 2 ** len(V)
    return {
        "n_vars": len(V), "runs": runs, "hits": hits,
        "success_rate": sr, "avg_flips_if_hit": (sum(flips)/len(flips)) if flips else None,
        "est_count_naive": sr * two_n, "method": "samplesat_runs"
    }

# --------- Comparative Table: random vs SampleSAT ---------
def compare_estimators(formulas,  # list of (name, sympy_expr)
                       trials_random=5000,
                       runs_samplesat=200,
                       noise=0.5,
                       rng=None):
    """
    Build a comparison over multiple formulas.
    Returns a list of dict rows with:
      - name, n_vars
      - random: p_hat, p_ci, est_count, est_ci, trials
      - samplesat: success_rate, est_count_naive, runs, avg_flips_if_hit
    """
    rng = rng or random.Random()
    rows = []
    for name, F in formulas:
        r = estimate_count_random(F, trials=trials_random, rng=rng)
        s = estimate_count_samplesat(F, runs=runs_samplesat, noise=noise, rng=rng)
        rows.append({
            "name": name,
            "n_vars": r["n_vars"],
            "random_trials": r["trials"],
            "p_hat": r["p_hat"],
            "p_ci_lo": r["p_ci"][0],
            "p_ci_hi": r["p_ci"][1],
            "est_count_random": r["est_count"],
            "est_ci_lo": r["est_ci"][0],
            "est_ci_hi": r["est_ci"][1],
            "samplesat_runs": s["runs"],
            "success_rate": s["success_rate"],
            "avg_flips_if_hit": s["avg_flips_if_hit"],
            "est_count_samplesat": s["est_count_naive"]
        })
    return rows

def compare_estimators_df(formulas, trials_random=5000, runs_samplesat=200, noise=0.5, rng=None):
    try:
        import pandas as pd
    except Exception:
        raise RuntimeError("pandas non disponibile: usa compare_estimators(...) per avere una lista di dict.")
    rows = compare_estimators(formulas, trials_random, runs_samplesat, noise, rng)
    cols = ["name","n_vars",
            "random_trials","p_hat","p_ci_lo","p_ci_hi",
            "est_count_random","est_ci_lo","est_ci_hi",
            "samplesat_runs","success_rate","avg_flips_if_hit","est_count_samplesat"]
    return pd.DataFrame(rows)[cols]

from sympy import symbols, And, Or, Not
import random

A,B,C,D = symbols('A B C D')
formule = [
    ("(A ∨ B)", Or(A,B)),
    ("XOR(A,B) ∧ C", And(Or(And(A, Not(B)), And(Not(A), B)), C)),
    ("(A→B) ∧ (C∨D)", And(Or(Not(A), B), Or(C, D))),
]

rng = random.Random(7)
df = compare_estimators_df(formule, trials_random=4000, runs_samplesat=200, noise=0.5, rng=rng)
print(df)

            name  n_vars  random_trials    p_hat   p_ci_lo   p_ci_hi  \
0        (A ∨ B)       2           4000  0.74575  0.732024  0.759004   
1   XOR(A,B) ∧ C       3           4000  0.24875  0.235599  0.262384   
2  (A→B) ∧ (C∨D)       4           4000  0.55975  0.544316  0.575070   

   est_count_random  est_ci_lo  est_ci_hi  samplesat_runs  success_rate  \
0             2.983   2.928097   3.036017             200           1.0   
1             1.990   1.884789   2.099069             200           1.0   
2             8.956   8.709053   9.201112             200           1.0   

   avg_flips_if_hit  est_count_samplesat  
0             1.285                  4.0  
1             1.950                  8.0  
2             1.545                 16.0  
